In [13]:
from o2o import BivariateHawkesProcessSimulator, ParameterEstimator, O2OAnalyzer
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("combined_data.csv", low_memory=False, na_values=['', ' '])
cols_to_drop = ["OFFENSE_CODE_GROUP", "UCR_PART"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")
df = df.dropna(subset=["Location", "DISTRICT"])
print(f"Cleaned dataset loaded into df, shape = {df.shape}")

Cleaned dataset loaded into df, shape = (410002, 15)


In [15]:
# Steven's function to put data in correct format
def parse_dates_multi_utc(col: pd.Series) -> pd.Series:
    s = col.astype(str).str.strip()
    s = (s.str.replace(r"[…]|\.{3}", "", regex=True)
           .str.replace(r"\s+", " ", regex=True))

    # parse with utc=True every time so results are homogeneous tz-aware
    parsed = pd.to_datetime(s, format="%Y-%m-%d", errors="coerce", utc=True)

    for fmt in [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %I:%M %p",
        "%m/%d/%Y",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
    ]:
        mask = parsed.isna()
        if mask.any():
            parsed.loc[mask] = pd.to_datetime(s[mask], format=fmt, errors="coerce", utc=True)

    # final fallback (dateutil)
    mask = parsed.isna()
    if mask.any():
        parsed.loc[mask] = pd.to_datetime(s[mask], errors="coerce", utc=True)

    return parsed  # dtype should be datetime64[ns, UTC]

In [19]:
df_new = df.copy()

cutoff1 = pd.Timestamp('2025-04-01')
cutoff2 = pd.Timestamp('2025-10-01') 

df_new['OCCURRED_ON_DATE'] = parse_dates_multi_utc(df_new['OCCURRED_ON_DATE'])
df_new['OCCURRED_ON_DATE'] = df_new['OCCURRED_ON_DATE'].dt.tz_convert('UTC').dt.tz_localize(None)

df_timewindow = df_new[
        (df_new['OCCURRED_ON_DATE'] >= cutoff1) &
        (df_new['OCCURRED_ON_DATE'] <= cutoff2)
    ].copy()

most_occured_crime =df_timewindow['OFFENSE_DESCRIPTION'].value_counts()
print(most_occured_crime)


OFFENSE_DESCRIPTION
INVESTIGATE PERSON                            4463
SICK ASSIST                                   3456
M/V - LEAVING SCENE - PROPERTY DAMAGE         2125
LARCENY SHOPLIFTING                           1991
TOWED MOTOR VEHICLE                           1811
                                              ... 
DRUNKENNESS                                      2
PROSTITUTION - SOLICITING                        2
POSSESSION OF BURGLARIOUS TOOLS                  1
CHILD REQUIRING ASSISTANCE (FOMERLY CHINS)       1
PRISONER - SUICIDE / SUICIDE ATTEMPT             1
Name: count, Length: 111, dtype: int64


In [5]:
category_column = df['OFFENSE_DESCRIPTION']
unique_offenses = category_column.dropna().unique()

M = 1 
df0 = df.copy()

cutoff1 = pd.Timestamp('2025-04-01')
cutoff2 = pd.Timestamp('2025-10-01')  

offense_list = []
spillover_list = []
decay_list = []
percentages_list = []
base_list = []
#Iterating through each unique crime type (note the second district says D4, but was set to equal C11 (correct district)
for offense in unique_offenses:
    print(offense)
    df_new = df0.loc[df0["OFFENSE_DESCRIPTION"] == offense].copy()
    if df_new.empty:
        continue

    df_new['OCCURRED_ON_DATE'] = parse_dates_multi_utc(df_new['OCCURRED_ON_DATE'])
    df_new['OCCURRED_ON_DATE'] = df_new['OCCURRED_ON_DATE'].dt.tz_convert('UTC').dt.tz_localize(None)

    df_timewindow = df_new[
        (df_new['OCCURRED_ON_DATE'] >= cutoff1) &
        (df_new['OCCURRED_ON_DATE'] <= cutoff2)
    ].copy()
    if df_timewindow.empty:
        continue

    B2 = df_timewindow[df_timewindow['DISTRICT'] == 'B2'].copy()
    D4 = df_timewindow[df_timewindow['DISTRICT'] == 'C11'].copy()
    if B2.empty or D4.empty:
        continue

    t0_B2 = B2['OCCURRED_ON_DATE'].min()
    t0_D4 = D4['OCCURRED_ON_DATE'].min()
    if pd.isna(t0_B2) or pd.isna(t0_D4):
        continue

    span_days = max(
        (B2['OCCURRED_ON_DATE'].max() - B2['OCCURRED_ON_DATE'].min()).days,
        (D4['OCCURRED_ON_DATE'].max() - D4['OCCURRED_ON_DATE'].min()).days
    )
    T = max(span_days, 1)


    B2['t'] = (B2['OCCURRED_ON_DATE'] - t0_B2).dt.total_seconds() / 86400.0
    D4['t'] = (D4['OCCURRED_ON_DATE'] - t0_D4).dt.total_seconds() / 86400.0

    data_B2 = [sorted(B2['t'].dropna().tolist())]
    data_D4 = [sorted(D4['t'].dropna().tolist())]
    
    #O2O model
    estimator = ParameterEstimator(data_B2, data_D4, M, T)
    fit = estimator.fit_model()
    spillover, decay = estimator.spillover_and_decays_values_and_CI()
    percentages = estimator.spillover_percentage()
    base_estimation = estimator.base_and_CI()

    offense_list.append(offense)
    spillover_list.append(spillover)
    decay_list.append(decay)
    percentages_list.append(percentages)
    base_list.append(base_estimation)


FRAUD - FALSE PRETENSE / SCHEME


12:33:08 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

12:33:16 - cmdstanpy - INFO - CmdStan done processing.



FRAUD - IMPERSONATION


...


Justifiable Homicide
TRUANCY / RUNAWAY
POSSESSION OF BURGLARIOUS TOOLS
EXPLOSIVES - TURNED IN OR FOUND
OBSCENE PHONE CALLS
PROSTITUTION - ASSISTING OR PROMOTING
Evidence Tracker Incidents
LARCENY THEFT FROM COIN-OP MACHINE
MANSLAUGHTER - NEGLIGENCE
FIREARM/WEAPON - LOST
MANSLAUGHTER - VEHICLE - NEGLIGENCE
FIREARM/WEAPON - ACCIDENTAL INJURY / DEATH
MURDER, NON-NEGLIGENT MANSLAUGHTER


14:52:36 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

14:52:36 - cmdstanpy - INFO - CmdStan done processing.



PRISONER ESCAPE / ESCAPE & RECAPTURE


In [6]:
print(len(spillover_list))
print(len(offense_list))

85
85


In [7]:
print(spillover_list)

[                         Effect  Value          95% CI
0    Online to Online Spillover  0.081  [0.003, 0.269]
1  Offline to Offline Spillover  0.227  [0.041, 0.533]
2   Online to Offline Spillover  0.089  [0.005, 0.254]
3   Offline to Online Spillover  0.052  [0.003, 0.137],                          Effect  Value          95% CI
0    Online to Online Spillover  0.267  [0.018, 0.721]
1  Offline to Offline Spillover  0.130  [0.013, 0.318]
2   Online to Offline Spillover  0.231  [0.013, 0.640]
3   Offline to Online Spillover  0.083  [0.004, 0.262],                          Effect  Value          95% CI
0    Online to Online Spillover  0.133  [0.009, 0.357]
1  Offline to Offline Spillover  0.088  [0.006, 0.241]
2   Online to Offline Spillover  0.128  [0.005, 0.418]
3   Offline to Online Spillover  0.040  [0.002, 0.128],                          Effect  Value          95% CI
0    Online to Online Spillover  0.143  [0.008, 0.428]
1  Offline to Offline Spillover  0.157  [0.017, 0.408]
2   On

In [8]:
roxbury_to_dorchester= []
dorchester_to_roxbury = []
CI_RD = []
CI_DR =[]
#getting the values and CI for the spillover 
for df in spillover_list:
    value1 = df.loc[2, 'Value'] 
    value2 = df.loc[3, 'Value'] 
    CI_value1 = df.loc[2, "95% CI"] 
    CI_value2 = df.loc[3, "95% CI"] 
    roxbury_to_dorchester.append(value1)
    dorchester_to_roxbury.append(value2)
    CI_RD.append(CI_value1)
    CI_DR.append(CI_value2)

#get the CI in correct format
CI_cleaned1 =[]

for row in CI_RD:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned1.append(numbers)

CI_cleaned2 =[]

for row in CI_DR:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned2.append(numbers)

    

In [11]:
# Making dataframe of values (unsorted/sorted)

roxbury_to_dorchester = [float(s) for s in roxbury_to_dorchester]
dorchester_to_roxbury = [float(s) for s in dorchester_to_roxbury]

data = {'Offense': offense_list, 'Spillover Roxbury to Dorchester': roxbury_to_dorchester, 'Spillover Dorchester to Roxbury':dorchester_to_roxbury}

data2 = {'Offense': offense_list, 'Spillover Roxbury to Dorchester': roxbury_to_dorchester}
data3 = {'Offense': offense_list,'Spillover Dorchester to Roxbury':dorchester_to_roxbury}


spilloverdf = pd.DataFrame(data)
spilloverdf2 = pd.DataFrame(data2)
spilloverdf3 = pd.DataFrame(data3)


spilloverdf['CI 95% Roxbury to Dorchester'] = CI_cleaned1
spilloverdf['CI 95% Dorchester to Roxbury'] = CI_cleaned2
spilloverdf2['CI 95% Roxbury to Dorchester'] = CI_cleaned1
spilloverdf3['CI 95% Dorchester to Roxbury'] = CI_cleaned2


print(spilloverdf)
sorted_spilloverdf2 = spilloverdf2.sort_values(by='Spillover Roxbury to Dorchester', ascending=False)
sorted_spilloverdf3 = spilloverdf3.sort_values(by='Spillover Dorchester to Roxbury', ascending=False)

spilloverdf.to_csv('\O2O\O2O\crimespilloverDorchester_notsorted.csv')
sorted_spilloverdf2.to_csv('\O2O\O2O\crimespilloverDorchester_sortedroxbury.csv')
sorted_spilloverdf3.to_csv('\O2O\O2O\crimespilloverDorchester_sortedDorchester.csv')





                                              Offense  \
0                     FRAUD - FALSE PRETENSE / SCHEME   
1                               FRAUD - IMPERSONATION   
2                       SICK/INJURED/MEDICAL - PERSON   
3                            FORGERY / COUNTERFEITING   
4                         LARCENY THEFT FROM BUILDING   
..                                                ...   
80                                              ARSON   
81                                        BOMB THREAT   
82          OPERATING UNDER THE INFLUENCE (OUI) DRUGS   
83  RECOVERED - MV RECOVERED IN BOSTON (STOLEN IN ...   
84                 MURDER, NON-NEGLIGENT MANSLAUGHTER   

    Spillover Roxbury to Dorchester  Spillover Dorchester to Roxbury  \
0                             0.089                            0.052   
1                             0.231                            0.083   
2                             0.128                            0.040   
3                          

In [10]:
import os
current_directory = os.getcwd()
print(f"Current Working Directory: {current_directory}")

Current Working Directory: C:\O2O\O2O
